In [58]:
!pip install loguru

In [59]:
import argparse
from collections import namedtuple, defaultdict
from csv import DictWriter
import json
from pathlib import Path

import requests
from loguru import logger

In [60]:
columns = ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 'description', 'engineDisplacement',
           'enginePower', 'location',
           'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 'name',
           'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen',
           'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС', 'Привод',
           'Руль', 'Состояние', 'Таможня', 'price']

In [61]:
CarInfo = namedtuple('Car', columns)

In [62]:
brand_in_test = {
    'BMW': 4473,
    'VOLKSWAGEN': 4404,
    'NISSAN': 4393,
    'MERCEDES': 4180,
    'TOYOTA': 3913,
    'AUDI': 3421,
    'MITSUBISHI': 2843,
    'SKODA': 2741,
    'VOLVO': 1463,
    'HONDA': 1150,
    'INFINITI': 871,
    'LEXUS': 834
}

In [63]:
cars_in_test = sum(val for val in brand_in_test.values())
print(cars_in_test)

34686


In [64]:
proportion_in_test = {key: val / cars_in_test for key, val in brand_in_test.items()}
print(proportion_in_test)

{'BMW': 0.12895692786715102, 'VOLKSWAGEN': 0.12696765265524995, 'NISSAN': 0.12665052182436717, 'MERCEDES': 0.12050971573545523, 'TOYOTA': 0.11281208556766419, 'AUDI': 0.09862768840454363, 'MITSUBISHI': 0.08196390474543043, 'SKODA': 0.07902323704088104, 'VOLVO': 0.04217840050740933, 'HONDA': 0.033154586865017585, 'INFINITI': 0.02511099579080897, 'LEXUS': 0.02404428299602145}


In [65]:
def get_headers():
    headers = '''
Host: auto.ru
Connection: keep-alive
Content-Length: 99
x-requested-with: fetch
x-client-date: 1603066469874
x-csrf-token: c23073bb4cd65413662a41bd460fd8317459fe3ce6d83db1
x-page-request-id: 3c4800b60eb9e8c568e5a515f5cd4872
content-type: application/json
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36
x-client-app-version: 202010.16.122434
Accept: */*
Origin: https://auto.ru
Sec-Fetch-Site: same-origin
Sec-Fetch-Mode: same-origin
Sec-Fetch-Dest: empty
Referer: https://auto.ru/cars/bmw/all/?output_type=list&page=1
Accept-Encoding: gzip, deflate, br
Accept-Language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7
Cookie: autoru_sid=a%3Ag5f88f7d72b1bif6t0m99h0krigjgs2e.a783a85b576c8a7acaea4faafaa81ffc%7C1602811863959.604800.QhhqH0HWfM4BPWrsvjyfIg.8EXrpUR7Bq1a2gOSCnsb0HnXxvmHbYB9eF5Uz5o_bZE; autoruuid=g5f88f7d72b1bif6t0m99h0krigjgs2e.a783a85b576c8a7acaea4faafaa81ffc; suid=63abf8672f4a9e550bb96dd00b95ad21.8e155bfb838a5227bdd9dea1d2cbdc3e; _ym_uid=1602811867719278329; yuidcs=1; crookie=PZwS3/iYq2PFIw/dbrsqDVB/0e2v79Xe/8RsG6ySC8Djcl+mh/UCjYohgODaSkw7rMa6O9v7+RD56YSQKE2fhSkWxV8=; cmtchd=MTYwMjgyODMxMjMwNQ==; bltsr=1; yuidlt=1; yandexuid=360949521578055883; my=YwA%3D; _ym_isad=1; promo-app-banner-shown=1; promo-header-counter=4; _csrf_token=c23073bb4cd65413662a41bd460fd8317459fe3ce6d83db1; from=direct; X-Vertis-DC=myt; _ym_wasSynced=%7B%22time%22%3A1603066366683%2C%22params%22%3A%7B%22eu%22%3A0%7D%2C%22bkParams%22%3A%7B%7D%7D; gdpr=0; _ym_visorc_22753222=b; from_lifetime=1603066463083; _ym_d=1603066470'''
    headers = {line.split(': ')[0]: line.split(': ')[1] for line in headers.strip().split('\n')}
    return headers

In [66]:
def get_data_from_site(pange_num: int, brand: str) -> dict:
    """ получает данные о тачках БМВ с авто.ру, номер страницы передан в параметре page_num
    возвращает полученный с сайта json """

    base_url = 'https://auto.ru/-/ajax/desktop/listing/'
    params = dict(category='cars', section="all", output_type="list", page=pange_num,
                  catalog_filter=[{'mark': brand}], geo_id=[213], geo_radius=800)
    r = requests.post(base_url, json=params, headers=get_headers())
    r.raise_for_status()
    return r.json()

In [67]:
try:
    get_data_from_site(1,'BMW')['offers'][0]['documents']['purchase_date']
except KeyError:
    purchase_date = None

In [68]:
def add_car_data(car_data: list, data_json: dict):
    """ разбирает данные из переданного объекта data_json
    из массива offers, формирует объекты CarInfo и
    добавляет их в список car_data"""

    if 'offers' not in data_json:
        return 

    for car in data_json['offers']:
        if 'configuration' not in car['vehicle_info']:
            continue

        body_type_human = car['vehicle_info']['configuration']['human_name']
        body_type = car['vehicle_info']['configuration']['body_type']
        transmission = car['vehicle_info']['tech_param']['transmission']
        engine_volume = car['vehicle_info']['tech_param']['displacement']
        engine_volume = round(float(engine_volume) / 1000, 1)

        try:
            purchase_date = car['documents']['purchase_date']
        except KeyError:
            purchase_date = None

        try:
            owners_number = car['documents']['owners_number']
        except KeyError:
            owners_number = None

        try:
            description = car['description']
        except KeyError:
            description = None
        try:
            pts = car['documents']['pts']
        except KeyError:
            pts = None
        try:
            price = car['price_info']['RUR']
        except KeyError:
            continue

        brand = car['vehicle_info']['mark_info']['name']
        model = car['vehicle_info']['model_info']['name']
        sell_id = car['saleId']
        section = car['section']
        car_url = f'https://auto.ru/cars/{section}/sale/{brand.lower()}/{model.lower()}/{sell_id}/'

        info = CarInfo(bodyType=body_type_human,
                       brand=brand,
                       car_url=car_url,
                       image=car['state']['image_urls'][0]['sizes']['small'],
                       color=car['color_hex'],
                       complectation_dict=car['vehicle_info']['complectation'],
                       equipment_dict=car['vehicle_info']['equipment'],
                       model_info=car['vehicle_info']['model_info'],
                       model_name=model,
                       location=car['seller']['location']['region_info']['name'],
                       parsing_unixtime=car['additional_info']['fresh_date'],
                       priceCurrency=car['price_info']['currency'],
                       sell_id=sell_id,
                       super_gen=car['vehicle_info']['super_gen'],
                       vendor=car['vehicle_info']['vendor'],
                       fuelType=car['vehicle_info']['tech_param']['engine_type'],
                       modelDate=car['vehicle_info']['super_gen']['year_from'],
                       name=car['vehicle_info']['tech_param']['human_name'],
                       numberOfDoors=car['vehicle_info']['configuration']['doors_count'],
                       productionDate=car['documents']['year'],
                       vehicleConfiguration=body_type + " " + transmission + " " + str(engine_volume),
                       vehicleTransmission=transmission,
                       engineDisplacement=str(engine_volume) + ' LTR',
                       enginePower=str(car['vehicle_info']['tech_param']['power']) + ' N12',
                       description=description,
                       mileage=car['state']['mileage'],
                       Привод=car['vehicle_info']['tech_param']['gear_type'],
                       Руль=car['vehicle_info']['steering_wheel'],
                       Состояние=car['state']['state_not_beaten'],
                       Владельцы=owners_number,
                       ПТС=pts,
                       Таможня=car['documents']['custom_cleared'],
                       Владение=purchase_date,
                       price=price
                       )
        car_data.append(info)

In [69]:
def write_to_csv(car_data: list, output_folder_path: Path, page_num: int):
    """Записывает информацию о машинах из car_data в файл формата csv
    output_folder_path задает папку для сохранения
    page_num - нужен для наименования файла"""

    if not car_data:
        return

    filename = output_folder_path / f'train_{page_num}.csv'
    with open(filename, 'w', encoding='utf-8', newline='') as f:
        logger.info(f'writing {filename}, содержащий {len(car_data)} записей')
        writer = DictWriter(f, fieldnames=columns)
        writer.writeheader()
        writer.writerows([car._asdict() for car in car_data])

In [70]:
def pickup_brand(brand_stats: dict) -> str:
    cars_count = sum(val for val in brand_stats.values())
    brand_proportion = {key: val / cars_count for key, val in brand_stats.items()}

    proportion_in_test_copy = proportion_in_test.copy()
    for key, val in brand_proportion.items():
        proportion_in_test_copy[key] -= val

    return sorted(proportion_in_test_copy.items(),
                  key=lambda x: x[1],
                  reverse=True)[0][0] # first element, key of pair key:value

In [75]:
def parse_data(n_pages, output_folder, save_json, json_folder):
    """Входная точка в программу. Содержит верхнеуровневую логику.
    Собирает информацию с сайта, парсит и сохраняет в формате csv"""

    logger.info('Starting parse data from auto.ru about cars')

    if save_json:
        json_folder_path = Path(json_folder)
        json_folder_path.mkdir(parents=True, exist_ok=True)

    output_folder_path = Path(output_folder)
    output_folder_path.mkdir(parents=True, exist_ok=True)

    car_data = []
    brand_stats = defaultdict(lambda: 1)

    for page_num in range(1500, n_pages + 1):

        brand = pickup_brand(brand_stats)

        logger.info(f'processing page: {page_num} {brand} {brand_stats[brand]}')
        try:
            data_json = get_data_from_site(brand_stats[brand], brand)
        except Exception as e:
            logger.error(f"Error in parsing: {e}")
            continue

        brand_stats[brand] += 1

        if save_json:
            with open(json_folder_path / f'page_{page_num}.json', 'w', encoding='utf-8') as f:
                json.dump(data_json, f)

        add_car_data(car_data, data_json)

        if page_num % 50 == 0:
            write_to_csv(car_data, output_folder_path, page_num)
            car_data = []

    write_to_csv(car_data, output_folder_path, page_num)

    logger.info('parsing successfully finished')

In [72]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("n_pages", help="number of pages to parse", type=int)
    parser.add_argument("-o", "-train_folder", help='folder to save parsed csv files', type=str, default='.')
    parser.add_argument("-j", "--save_json", help="save json files", action="store_true")
    parser.add_argument("--json_folder", help='folder for json files', type=str, default='json')
    args = parser.parse_args()
    print(args)
    ##parse_data(args.n_pages,
    #           args.output_folder,
    #           args.save_json,
    #           args.json_folder)

In [73]:
if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [-o O] [-j] [--json_folder JSON_FOLDER] n_pages
ipykernel_launcher.py: error: argument n_pages: invalid int value: 'C:\\Users\\User-PC\\AppData\\Roaming\\jupyter\\runtime\\kernel-0674999f-5e03-4dbe-ba45-b670ca61e533.json'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\User-PC\anaconda3\lib\argparse.py", line 2422, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: 'C:\\Users\\User-PC\\AppData\\Roaming\\jupyter\\runtime\\kernel-0674999f-5e03-4dbe-ba45-b670ca61e533.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User-PC\anaconda3\lib\argparse.py", line 1800, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "C:\Users\User-PC\anaconda3\lib\argparse.py", line 2009, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "C:\Users\User-PC\anaconda3\lib\argparse.py", line 1965, in consume_positionals
    take_action(action, args)
  File "C:\Users\User-PC\anaconda3\lib\argparse.py", line 1858, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "C:\Users\User-PC\anaconda3\lib\argpars

TypeError: object of type 'NoneType' has no len()

In [76]:
parse_data(5000,'train_folder',1,'json_folder')

2021-03-06 13:12:51.350 | INFO     | __main__:parse_data:5 - Starting parse data from auto.ru about cars
2021-03-06 13:12:51.351 | INFO     | __main__:parse_data:21 - processing page: 1500 BMW 1
2021-03-06 13:12:52.086 | INFO     | __main__:write_to_csv:11 - writing train_folder\train_1500.csv, содержащий 37 записей
2021-03-06 13:12:52.095 | INFO     | __main__:parse_data:21 - processing page: 1501 VOLKSWAGEN 1
2021-03-06 13:12:52.702 | INFO     | __main__:parse_data:21 - processing page: 1502 NISSAN 1
2021-03-06 13:12:53.268 | INFO     | __main__:parse_data:21 - processing page: 1503 MERCEDES 1
2021-03-06 13:12:53.870 | INFO     | __main__:parse_data:21 - processing page: 1504 TOYOTA 1
2021-03-06 13:12:54.426 | INFO     | __main__:parse_data:21 - processing page: 1505 AUDI 1
2021-03-06 13:12:55.052 | INFO     | __main__:parse_data:21 - processing page: 1506 MITSUBISHI 1
2021-03-06 13:12:55.628 | INFO     | __main__:parse_data:21 - processing page: 1507 SKODA 1
2021-03-06 13:12:56.297 

2021-03-06 13:13:44.116 | INFO     | __main__:parse_data:21 - processing page: 1584 AUDI 9
2021-03-06 13:13:44.911 | INFO     | __main__:parse_data:21 - processing page: 1585 BMW 12
2021-03-06 13:13:45.583 | INFO     | __main__:parse_data:21 - processing page: 1586 VOLKSWAGEN 12
2021-03-06 13:13:46.227 | INFO     | __main__:parse_data:21 - processing page: 1587 NISSAN 12
2021-03-06 13:13:46.786 | INFO     | __main__:parse_data:21 - processing page: 1588 LEXUS 2
2021-03-06 13:13:47.581 | INFO     | __main__:parse_data:21 - processing page: 1589 TOYOTA 11
2021-03-06 13:13:48.299 | INFO     | __main__:parse_data:21 - processing page: 1590 HONDA 3
2021-03-06 13:13:49.008 | INFO     | __main__:parse_data:21 - processing page: 1591 MITSUBISHI 8
2021-03-06 13:13:49.506 | INFO     | __main__:parse_data:21 - processing page: 1592 MERCEDES 12
2021-03-06 13:13:50.286 | INFO     | __main__:parse_data:21 - processing page: 1593 BMW 13
2021-03-06 13:13:50.843 | INFO     | __main__:parse_data:21 - pr

2021-03-06 13:14:39.928 | INFO     | __main__:parse_data:21 - processing page: 1669 TOYOTA 20
2021-03-06 13:14:40.449 | INFO     | __main__:parse_data:21 - processing page: 1670 BMW 23
2021-03-06 13:14:41.051 | INFO     | __main__:parse_data:21 - processing page: 1671 SKODA 14
2021-03-06 13:14:41.688 | INFO     | __main__:parse_data:21 - processing page: 1672 LEXUS 4
2021-03-06 13:14:42.436 | INFO     | __main__:parse_data:21 - processing page: 1673 VOLKSWAGEN 23
2021-03-06 13:14:42.941 | INFO     | __main__:parse_data:21 - processing page: 1674 NISSAN 23
2021-03-06 13:14:43.415 | INFO     | __main__:parse_data:21 - processing page: 1675 MERCEDES 22
2021-03-06 13:14:43.948 | INFO     | __main__:parse_data:21 - processing page: 1676 AUDI 18
2021-03-06 13:14:44.571 | INFO     | __main__:parse_data:21 - processing page: 1677 MITSUBISHI 15
2021-03-06 13:14:45.142 | INFO     | __main__:parse_data:21 - processing page: 1678 BMW 24
2021-03-06 13:14:45.740 | INFO     | __main__:parse_data:21 -

2021-03-06 13:15:28.121 | INFO     | __main__:parse_data:21 - processing page: 1754 LEXUS 6
2021-03-06 13:15:28.755 | INFO     | __main__:parse_data:21 - processing page: 1755 BMW 34
2021-03-06 13:15:29.491 | INFO     | __main__:parse_data:21 - processing page: 1756 AUDI 26
2021-03-06 13:15:29.892 | INFO     | __main__:parse_data:21 - processing page: 1757 MERCEDES 32
2021-03-06 13:15:30.473 | INFO     | __main__:parse_data:21 - processing page: 1758 TOYOTA 30
2021-03-06 13:15:30.896 | INFO     | __main__:parse_data:21 - processing page: 1759 VOLVO 11
2021-03-06 13:15:31.355 | INFO     | __main__:parse_data:21 - processing page: 1760 VOLKSWAGEN 34
2021-03-06 13:15:31.786 | INFO     | __main__:parse_data:21 - processing page: 1761 NISSAN 34
2021-03-06 13:15:32.229 | INFO     | __main__:parse_data:21 - processing page: 1762 SKODA 21
2021-03-06 13:15:32.833 | INFO     | __main__:parse_data:21 - processing page: 1763 MITSUBISHI 22
2021-03-06 13:15:33.303 | INFO     | __main__:parse_data:21

2021-03-06 13:16:10.558 | INFO     | __main__:parse_data:21 - processing page: 1840 NISSAN 44
2021-03-06 13:16:11.176 | INFO     | __main__:parse_data:21 - processing page: 1841 MERCEDES 42
2021-03-06 13:16:11.719 | INFO     | __main__:parse_data:21 - processing page: 1842 BMW 45
2021-03-06 13:16:12.226 | INFO     | __main__:parse_data:21 - processing page: 1843 LEXUS 8
2021-03-06 13:16:12.831 | INFO     | __main__:parse_data:21 - processing page: 1844 VOLKSWAGEN 45
2021-03-06 13:16:13.195 | INFO     | __main__:parse_data:21 - processing page: 1845 TOYOTA 40
2021-03-06 13:16:13.616 | INFO     | __main__:parse_data:21 - processing page: 1846 MITSUBISHI 29
2021-03-06 13:16:14.083 | INFO     | __main__:parse_data:21 - processing page: 1847 NISSAN 45
2021-03-06 13:16:14.460 | INFO     | __main__:parse_data:21 - processing page: 1848 AUDI 35
2021-03-06 13:16:14.842 | INFO     | __main__:parse_data:21 - processing page: 1849 BMW 46
2021-03-06 13:16:15.404 | INFO     | __main__:parse_data:21 

...

2021-03-06 13:44:00.841 | INFO     | __main__:parse_data:21 - processing page: 4629 MERCEDES 378
2021-03-06 13:44:01.566 | INFO     | __main__:parse_data:21 - processing page: 4630 MITSUBISHI 257
2021-03-06 13:44:02.442 | INFO     | __main__:parse_data:21 - processing page: 4631 TOYOTA 354
2021-03-06 13:44:03.020 | INFO     | __main__:parse_data:21 - processing page: 4632 SKODA 248
2021-03-06 13:44:03.592 | INFO     | __main__:parse_data:21 - processing page: 4633 BMW 405
2021-03-06 13:44:04.439 | INFO     | __main__:parse_data:21 - processing page: 4634 NISSAN 398
2021-03-06 13:44:05.131 | INFO     | __main__:parse_data:21 - processing page: 4635 VOLKSWAGEN 399
2021-03-06 13:44:05.839 | INFO     | __main__:parse_data:21 - processing page: 4636 AUDI 310
2021-03-06 13:44:06.386 | INFO     | __main__:parse_data:21 - processing page: 4637 MERCEDES 379
2021-03-06 13:44:07.001 | INFO     | __main__:parse_data:21 - processing page: 4638 HONDA 104
2021-03-06 13:44:07.669 | INFO     | __main__

2021-03-06 13:44:53.974 | INFO     | __main__:parse_data:21 - processing page: 4713 VOLKSWAGEN 409
2021-03-06 13:44:54.557 | INFO     | __main__:parse_data:21 - processing page: 4714 NISSAN 408
2021-03-06 13:44:55.277 | INFO     | __main__:parse_data:21 - processing page: 4715 MITSUBISHI 264
2021-03-06 13:44:55.879 | INFO     | __main__:parse_data:21 - processing page: 4716 AUDI 318
2021-03-06 13:44:56.689 | INFO     | __main__:parse_data:21 - processing page: 4717 BMW 416
2021-03-06 13:44:57.372 | INFO     | __main__:parse_data:21 - processing page: 4718 TOYOTA 364
2021-03-06 13:44:58.127 | INFO     | __main__:parse_data:21 - processing page: 4719 MERCEDES 389
2021-03-06 13:44:58.796 | INFO     | __main__:parse_data:21 - processing page: 4720 SKODA 255
2021-03-06 13:44:59.459 | INFO     | __main__:parse_data:21 - processing page: 4721 VOLKSWAGEN 410
2021-03-06 13:45:00.071 | INFO     | __main__:parse_data:21 - processing page: 4722 NISSAN 409
2021-03-06 13:45:00.701 | INFO     | __mai

2021-03-06 13:45:47.764 | INFO     | __main__:parse_data:21 - processing page: 4798 AUDI 326
2021-03-06 13:45:48.404 | INFO     | __main__:parse_data:21 - processing page: 4799 TOYOTA 373
2021-03-06 13:45:49.103 | INFO     | __main__:parse_data:21 - processing page: 4800 VOLKSWAGEN 420
2021-03-06 13:45:49.607 | INFO     | __main__:write_to_csv:11 - writing train_folder\train_4800.csv, содержащий 1776 записей
2021-03-06 13:45:49.883 | INFO     | __main__:parse_data:21 - processing page: 4801 NISSAN 419
2021-03-06 13:45:50.418 | INFO     | __main__:parse_data:21 - processing page: 4802 MITSUBISHI 271
2021-03-06 13:45:50.929 | INFO     | __main__:parse_data:21 - processing page: 4803 BMW 427
2021-03-06 13:45:51.665 | INFO     | __main__:parse_data:21 - processing page: 4804 MERCEDES 399
2021-03-06 13:45:52.334 | INFO     | __main__:parse_data:21 - processing page: 4805 INFINITI 83
2021-03-06 13:45:52.483 | INFO     | __main__:parse_data:21 - processing page: 4806 TOYOTA 374
2021-03-06 13:

2021-03-06 13:46:39.927 | INFO     | __main__:parse_data:21 - processing page: 4882 BMW 437
2021-03-06 13:46:40.635 | INFO     | __main__:parse_data:21 - processing page: 4883 SKODA 268
2021-03-06 13:46:41.230 | INFO     | __main__:parse_data:21 - processing page: 4884 MITSUBISHI 278
2021-03-06 13:46:41.789 | INFO     | __main__:parse_data:21 - processing page: 4885 MERCEDES 409
2021-03-06 13:46:42.325 | INFO     | __main__:parse_data:21 - processing page: 4886 VOLKSWAGEN 431
2021-03-06 13:46:42.863 | INFO     | __main__:parse_data:21 - processing page: 4887 NISSAN 430
2021-03-06 13:46:43.428 | INFO     | __main__:parse_data:21 - processing page: 4888 TOYOTA 383
2021-03-06 13:46:44.112 | INFO     | __main__:parse_data:21 - processing page: 4889 BMW 438
2021-03-06 13:46:44.678 | INFO     | __main__:parse_data:21 - processing page: 4890 AUDI 335
2021-03-06 13:46:45.456 | INFO     | __main__:parse_data:21 - processing page: 4891 VOLVO 143
2021-03-06 13:46:46.118 | INFO     | __main__:pars

2021-03-06 13:47:31.878 | INFO     | __main__:parse_data:21 - processing page: 4966 NISSAN 440
2021-03-06 13:47:32.534 | INFO     | __main__:parse_data:21 - processing page: 4967 TOYOTA 392
2021-03-06 13:47:33.125 | INFO     | __main__:parse_data:21 - processing page: 4968 INFINITI 87
2021-03-06 13:47:33.257 | INFO     | __main__:parse_data:21 - processing page: 4969 MERCEDES 419
2021-03-06 13:47:33.847 | INFO     | __main__:parse_data:21 - processing page: 4970 HONDA 115
2021-03-06 13:47:34.480 | INFO     | __main__:parse_data:21 - processing page: 4971 AUDI 343
2021-03-06 13:47:35.171 | INFO     | __main__:parse_data:21 - processing page: 4972 MITSUBISHI 285
2021-03-06 13:47:35.665 | INFO     | __main__:parse_data:21 - processing page: 4973 VOLKSWAGEN 442
2021-03-06 13:47:36.275 | INFO     | __main__:parse_data:21 - processing page: 4974 BMW 449
2021-03-06 13:47:37.052 | INFO     | __main__:parse_data:21 - processing page: 4975 NISSAN 441
2021-03-06 13:47:37.602 | INFO     | __main__